In [19]:
#coding:utf-8
#python3
import re
import string
import sys
import os
import urllib
from bs4 import BeautifulSoup
import requests
import time
from lxml import etree
import traceback
import pandas as pd 
import numpy as np 
from pandas import Series,DataFrame
import jieba
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
import gensim
import json
from gensim.models import word2vec
import logging
import codecs



class weibo:
    
    #weibo类初始化
    def __init__(self,user_id,filter = 0):
            self.user_id = user_id #用户id，即需要我们输入的数字，如昵称为“Dear-迪丽热巴”的id为1669879400
            self.filter = filter #取值范围为0、1，程序默认值为0，代表要爬取用户的全部微博，1代表只爬取用户的原创微博
            self.userName = '' #用户名，如“Dear-迪丽热巴”
            self.weiboNum = 0 #用户全部微博数
            self.weiboNum2 = 0 #爬取到的微博数
            self.following = 0 #用户关注数
            self.followers = 0 #用户粉丝数
            self.weibos = [] #微博内容
            self.num_zan = [] #微博对应的点赞数
            self.num_forwarding = [] #微博对应的转发数
            self.num_comment = [] #微博对应的评论数
            self.session = requests.session()
            self.weibos_flag = []
            self.headers2 = {
                "Accept":"*/*",
                "Accept-Encoding":"gzip, deflate, br",
                "Accept-Language":"en-US,en;q=0.8",
                "Connection":"keep-alive",
                "Content-Length":"184",
                "Content-Type":"application/x-www-form-urlencoded",
                "Host":"passport.weibo.cn",
                "Origin":"https://passport.weibo.cn",
                "Referer":"https://passport.weibo.cn/signin/login?entry=mweibo&r=http://weibo.cn/",
                "User-Agent":"Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/57.0.2987.133 Safari/537.36"
                }
            self.headers3 = {
                "Accept":"text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8",
                "Accept-Encoding":"gzip, deflate, sdch, br",
                "Accept-Language":"en-US,en;q=0.8",
                "Connection":"keep-alive",
                "Host":"weibo.cn",
                "Upgrade-Insecure-Requests":"1",
                "User-Agent":"Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/57.0.2987.133 Safari/537.36"
                }
            self.data = {
                "username":"727883459@qq.com",
                "password":"weibo@q5.com",
                "savestate":"1",
                "r":"http://weibo.cn/",
                "ec":"0",
                "pagerefer":"",
                "entry":"mweibo",
                "wentry":"",
                "loginfrom":"",
                "client_id":"",
                "code":"",
                "qq":"",
                "mainpageflag":"1",
                "hff":"",
                "hfp":""
                }
            self.time_list = []
    #获取用户昵称     

    def login(self):
        
        url = "https://passport.weibo.cn/sso/login"
        cont = self.session.post(url,headers=self.headers2,data=self.data)
        print ("log")

    def getUserName(self):
        try:
            url = 'http://weibo.cn/%d/info'%(self.user_id)
            html = self.session.get(url, headers=self.headers3).content
            selector = etree.HTML(html)
            userName = selector.xpath("//title/text()")[0]
            self.userName = userName[:-3].encode('gbk')
            #print '用户昵称：' + self.userName
        except Exception as e:        
            print ("Error: ",e) 
            traceback.print_exc()
        

        
        
        
        
        
        
    #获取用户微博数、关注数、粉丝数
    def getUserInfo(self):
      try:
        url = 'http://weibo.cn/u/%d?filter=%d&page=1'%(self.user_id,self.filter)
        html = self.session.get(url, headers=self.headers3).content
#         with open("test.txt",'wb') as filename:
#             filename.write(html)
        selector = etree.HTML(html)
        pattern = r"\d+\.?\d*"

        #微博数
        str_wb = selector.xpath("//div[@class='tip2']/span[@class='tc']/text()")[0]
        guid = re.findall(pattern, str_wb, re.S|re.M)   
        for value in guid:   
            num_wb = int(value)  
            break
        self.weiboNum = num_wb  
        #print '微博数: ' + str(self.weiboNum) 
  
        #关注数
        str_gz = selector.xpath("//div[@class='tip2']/a/text()")[0]
        guid = re.findall(pattern, str_gz, re.M)  
        self.following = int(guid[0])  
        #print '关注数: ' + str(self.following)

        #粉丝数
        str_fs = selector.xpath("//div[@class='tip2']/a/text()")[1]
        guid = re.findall(pattern, str_fs, re.M)  
        self.followers = int(guid[0]) 
        #print '粉丝数: ' + str(self.followers)
      except Exception as e:        
        print ("Error: ",e)
        traceback.print_exc()
        
    #获取用户微博内容及对应的点赞数、转发数、评论数    
    def getWeiboInfo(self):
      try:
        url = 'http://weibo.cn/u/%d?filter=%d&page=1'%(self.user_id,self.filter)
        html = self.session.get(url, headers=self.headers3).content
        selector = etree.HTML(html)        
        if selector.xpath('//input[@name="mp"]')==[]:
           pageNum = 1
        else:
           pageNum = (int)(selector.xpath('//input[@name="mp"]')[0].attrib['value'])
        pattern = r"\d+\.?\d*"
        for page in range(1,min(6,pageNum+1)):
          print('page:'+str(page))
          url2 = 'http://weibo.cn/u/%d?filter=%d&page=%d'%(self.user_id,self.filter,page)
          html2 = self.session.get(url2, headers=self.headers3).content
          selector2 = etree.HTML(html2)
          info = selector2.xpath("//div[@class='c']")
          with open('test_html.txt','wb') as filename:
                filename.write(html)
          soup = BeautifulSoup(html2)
          self.time_list =self.time_list + [x.string for x in  soup.find_all('span',class_='ct')]
          #print len(info)
          if len(info) > 3:
            for i in range(0,len(info)-2):
              self.weiboNum2 = self.weiboNum2 + 1
              #微博内容
              str_t = info[i].xpath("div/span[@class='ctt']")
              weibos = str_t[0].xpath('string(.)').encode('gbk','ignore')
                
              if len(info[i].xpath("div/span[@class='cmt']")) !=0:
                    self.weibos_flag.append(1)
              else:
                    self.weibos_flag.append(0)
              self.weibos.append(weibos)
              #print '微博内容：'+ weibos
              #点赞数
              str_zan = info[i].xpath("div/a/text()")[-4]
              guid = re.findall(pattern, str_zan, re.M)  
              num_zan = int(guid[0])
              self.num_zan.append(num_zan)
              #print '点赞数: ' + str(num_zan)
              #转发数
              forwarding = info[i].xpath("div/a/text()")[-3]
              guid = re.findall(pattern, forwarding, re.M)  
              num_forwarding = int(guid[0])
              self.num_forwarding.append(num_forwarding)              
              #print '转发数: ' + str(num_forwarding)
              #评论数
              comment = info[i].xpath("div/a/text()")[-2]
              guid = re.findall(pattern, comment, re.M)  
              num_comment = int(guid[0]) 
              self.num_comment.append(num_comment)
              #print '评论数: ' + str(num_comment)
        if self.filter == 0:
          print ('共'+str(self.weiboNum2)+'条微博')
        else:
          print (u'共'+str(self.weiboNum)+'条微博，其中'+str(self.weiboNum2)+'条为原创微博')
      except Exception as e:        
        print ("Error: ",e)
        traceback.print_exc()
    
    #主程序
    def start(self):
      try:
        weibo.login(self)
        weibo.getUserName(self)
        weibo.getUserInfo(self)
        weibo.getWeiboInfo(self)
        print ('信息抓取完毕')
        print ('===========================================================================')
      except Exception as e:        
        print ("Error: ",e)
    
    #将爬取的信息写入文件 
    def writeTxt(self):
      txt_dict = {}
      try:
        if self.filter == 1:
            
           resultHeader = '\n\n原创微博内容：\n'
        else:
           resultHeader = '\n\n微博内容：\n'
        txt_dict['name'] =  self.userName.decode('gbk') #用户信息
        txt_dict['id'] = str(self.user_id)   #用户id
        txt_dict['weibos_num'] = str(self.weiboNum)   #微博数
        txt_dict['following_num'] = str(self.following)  #关注数
        txt_dict['followers_num'] = str(self.followers)   #粉丝数
        txt_dict['weibo'] = []

#         result = '用户信息\n用户昵称：' + self.userName.decode('gbk') + '\n用户id：' + str(self.user_id) + '\n微博数：' + str(self.weiboNum) + '\n关注数：' + str(self.following) + '\n粉丝数：' + str(self.followers) + resultHeader
        for i in range(1,self.weiboNum2 + 1):
            weibo_dict = {}
            weibo_dict['content'] = self.weibos[i-1].decode('gbk') #微博内容
            weibo_dict['upvote'] = str(self.num_zan[i-1]) #点赞数
            weibo_dict['forward'] = str(self.num_forwarding[i-1]) #转发数
            weibo_dict['comment'] = str(self.num_comment[i-1])  #评论数
            weibo_dict['time'] = self.time_list[i-1] #时间
            weibo_dict['forward_flag'] = self.weibos_flag[i-1] #是否是转发
            txt_dict['weibo'].append(weibo_dict)
        
#         text=str(i) + ':' + self.weibos[i-1].decode('gbk') + '\n'+'点赞数：' + str(self.num_zan[i-1]) + '    转发数：' + str(self.num_forwarding[i-1]) + '   评论数：' + str(self.num_comment[i-1]) + '\n'+self.time_list[i-1]+'\n'+'转发:'+str(self.weibos_flag[i-1])+'\n\n'
#         result = result + text
                                        
        result = json.dumps(txt_dict,sort_keys=True, indent=4,ensure_ascii=False)                           
        if os.path.isdir('weibo') == False:
           os.mkdir('weibo')
    
        f = open("weibo/%s.txt"%self.user_id, "w",encoding='utf-8')
        f.write(result)
        f.close()

    
        file_path=os.getcwd()+"\weibo"+"\%d"%self.user_id+".txt"
        print ('微博写入文件完毕，保存路径%s'%(file_path))
      except Exception as e:        
        print ("Error: ",e )
        traceback.print_exc()       
        

# def spider_run(user_id):
user_id = int(2161663171)
#可以改成任意合法的用户id（爬虫的微博id除外）
filter = 0 #值为0表示爬取全部的微博信息（原创微博+转发微博），值为1表示只爬取原创微博
wb = weibo(user_id,filter) #调用weibo类，创建微博实例wb
wb.start() #爬取微博信息
print ('用户名：' + wb.userName.decode('gbk'))
print('id:'+str(user_id))
print ('全部微博数：' + str(wb.weiboNum))
try:
    if wb.weiboNum !=0:
        print ('关注数：' + str(wb.following))
        print ('粉丝数：' + str(wb.followers))
        print ('最新一条微博为：' + wb.weibos[0].decode('gbk') )#若filter=1则为最新的原创微博，如果该用户微博数为0，即len(wb.weibos)==0,打印会出错，下同
        print ('最新一条微博获得的点赞数：' + str(wb.num_zan[0]))
        print ('最新一条微博获得的转发数：' + str(wb.num_forwarding[0]))
        print ('最新一条微博获得的评论数：' + str(wb.num_comment[0]))
    wb.writeTxt() #wb.writeTxt()只是把信息写到文件里，大家可以根据自己的需要重新编写writeTxt()函数
except Exception as e:        
    print ("Error: ",e )
    traceback.print_exc()    

# num  = 0
# user_post = pd.read_excel('../data/spammer_order.xls')['user_id']
# path = os.getcwd()
# file_list = os.listdir(path+'/weibo/')
# for i in user_post:
# 	id_num = str(i)
# 	filename = id_num+'.txt'
# 	if filename not in file_list:
# 		spider_run(int(i))
# 		time.sleep(10)
# 	num += 1
# 	print(num)
	

# spider_run(int(2161663171))

log
page:1


/usr/local/lib/python3.5/dist-packages/bs4/__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 184 of the file /usr/lib/python3.5/runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "lxml")

  markup_type=markup_type))


page:2
page:3
page:4
page:5
共46条微博
信息抓取完毕
用户名：暖暖学长zz
id:2161663171
全部微博数：7561
关注数：868
粉丝数：210
最新一条微博为：他们说我手机界面又脏又乱？？？难道大家不都这样吗？？？？ 
最新一条微博获得的点赞数：0
最新一条微博获得的转发数：0
最新一条微博获得的评论数：0
Error:  list index out of range


Traceback (most recent call last):
  File "<ipython-input-19-bf6e20ade83c>", line 240, in writeTxt
    weibo_dict['time'] = self.time_list[i-1] #时间
IndexError: list index out of range


In [59]:
with open('test_html.txt','rb') as filename:
    a = filename.readlines()[0]
    
soup = BeautifulSoup(a)
# print(soup)
# print(soup.find_all('span',class_='ct'))
b = soup.find_all('span',class_='ct')
b[0]

b[0].contents[0]
time_list = []
time_list =time_list + [x.contents[0] for x in  soup.find_all('span',class_='ct')]
time_list

/usr/local/lib/python3.5/dist-packages/bs4/__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 184 of the file /usr/lib/python3.5/runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "lxml")

  markup_type=markup_type))


<span class="ct">05月16日 22:55 来自<a href="https://weibo.cn/sinaurl?f=w&amp;u=http%3A%2F%2Fdown.sina.cn%2Fsinaclient%2Fweibo%2Findex%2Fgsid%2F0%2Fmid%2F0">Android客户端</a></span>

'05月16日 22:55\xa0来自'

['05月16日 22:55\xa0来自',
 '05月14日 19:52\xa0来自',
 '05月06日 00:20\xa0来自',
 '05月06日 00:19\xa0来自',
 '04月26日 18:39\xa0来自',
 '04月23日 09:38\xa0来自',
 '04月22日 10:00\xa0来自黄油相机',
 '04月17日 20:00\xa0来自',
 '04月12日 01:00\xa0来自']